In [12]:
import requests
import datatable as dt 
import pandas as pd 

from bs4 import BeautifulSoup

import os

df = dt.fread("monash-repository.csv")
df.head()

,Dataset,Domain,No: of Series,Min. Length,Max. Length,Competition,Multivariate,Download,Source,URL,Frequency
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,M1,Multiple,1001,15,150,Yes,No,Yearly,"Makridakis et al., 1982",https://zenodo.org/record/4656193,Yearly
1,M1,Multiple,1001,15,150,Yes,No,Quarterly,"Makridakis et al., 1982",https://zenodo.org/record/4656154,Quarterly
2,M1,Multiple,1001,15,150,Yes,No,Monthly,"Makridakis et al., 1982",https://zenodo.org/record/4656159,Monthly
3,M3,Multiple,3003,20,144,Yes,No,Yearly,"Makridakis and Hibon, 2000",https://zenodo.org/record/4656222,Yearly
4,M3,Multiple,3003,20,144,Yes,No,Quarterly,"Makridakis and Hibon, 2000",https://zenodo.org/record/4656262,Quarterly
5,M3,Multiple,3003,20,144,Yes,No,Monthly,"Makridakis and Hibon, 2000",https://zenodo.org/record/4656298,Monthly
6,M3,Multiple,3003,20,144,Yes,No,Other,"Makridakis and Hibon, 2000",https://zenodo.org/record/4656335,Other
7,M4,Multiple,100000,19,9933,Yes,No,Yearly,"Makridakis et al., 2020",https://zenodo.org/record/4656379,Yearly
8,M4,Multiple,100000,19,9933,Yes,No,Quarterly,"Makridakis et al., 2020",https://zenodo.org/record/4656410,Quarterly


In [2]:
df = df.to_pandas()

In [3]:
x = df['URL'][0]
x

'https://zenodo.org/record/4656193'

In [4]:
# Create Soup
page = requests.get(x)
soup = BeautifulSoup(page.text, 'html.parser')

In [5]:
info = soup.select("[class~=filename]")
[i.get('href') for i in info]

['/record/4656193/files/m1_yearly_dataset.zip?download=1']

In [6]:
check = info[0]
check.get('href')

'/record/4656193/files/m1_yearly_dataset.zip?download=1'

In [7]:
import urllib.request

# Download File
urllib.request.urlretrieve("https://zenodo.org" + check.get('href'), "check.zip")

('check.zip', <http.client.HTTPMessage at 0x7fc8d0026e20>)

In [8]:

# Parameters
# full_file_path_and_name - complete .tsf file path
# replace_missing_vals_with - a term to indicate the missing values in series in the returning dataframe
# value_column_name - Any name that is preferred to have as the name of the column containing series values in the returning dataframe
def convert_tsf_to_dataframe(
    full_file_path_and_name,
    replace_missing_vals_with="NaN",
    value_column_name="series_value",
):
    '''Converts the contents in a .tsf file into a dataframe and returns it along with other meta-data of the dataset: frequency, horizon, whether the dataset contains missing values and whether the series have equal lengths'''
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, "r", encoding="cp1252") as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"):  # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (
                                len(line_content) != 3
                            ):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if (
                                len(line_content) != 2
                            ):  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(
                                    strtobool(line_content[1])
                                )
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception(
                                "Missing attribute section. Attribute section must come before data."
                            )

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception(
                            "Missing attribute section. Attribute section must come before data."
                        )
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if len(series) == 0:
                            raise Exception(
                                "A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol"
                            )

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if numeric_series.count(replace_missing_vals_with) == len(
                            numeric_series
                        ):
                            raise Exception(
                                "All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series."
                            )

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(
                                    full_info[i], "%Y-%m-%d %H-%M-%S"
                                )
                            else:
                                raise Exception(
                                    "Invalid attribute type."
                                )  # Currently, the code supports only numeric, string and date types. Extend this as required.

                            if att_val is None:
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return (
            loaded_data,
            frequency,
            forecast_horizon,
            contain_missing_values,
            contain_equal_length,
        )

def parse_monash_df(file):
    
    ''' Function to Parse a Locally Extracted and Downloaded File'''
    
    loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe(file)

    parsed_df = pd.DataFrame()

    #freq = frequency
    if frequency == 'yearly':
        freq = 'YS' #year start
    elif frequency == 'quarterly':
        freq = 'QS' #quarter start
    elif frequency == 'monthly':
        freq = 'MS'
    elif frequency == 'daily':
        freq = 'D'


    for index,row in tqdm(loaded_data.iterrows()):
        
        name = row.series_name
        #print(name)
        values = row.series_value.tolist()
        length = len(values)
        start = row.start_timestamp
        #print(length)
        ds = pd.date_range(start, periods=length, freq=freq)
        
        series_df = pd.DataFrame({'unique_id':name,'ds':ds, 'values':values})
        parsed_df = pd.concat([parsed_df, series_df], axis=0)
        
    return parsed_df

# Example Usage
#parse_monash_df("m1_yearly_dataset.tsf").head()

def extract_monash_df(archive_file, origindestination):
    ''' Function to unzip and extract downloaded archive file'''   

    os.chdir(destination)

    dest = 'staging'
    print(dest)
    orig = os.getcwd()
    print(orig)

    # Create Staging Directory to Extract Archive File
    os.makedirs(dest, exist_ok=True)
    
    # Copy Archive File into Staging Directory
    shutil.copy(archive_file, dest)
    
    # Go into the directory and unpack
    os.chdir(dest)
    shutil.unpack_archive(archive_file
                          )
    # Find any TSF Files
    result = glob.glob('*.{}'.format('tsf'))
    #print(result[0])
    
    # Find the name of the file 
    local_name = result[0].split(".")[0]
    
    # Parse the DataFrame
    local_df = parse_monash_df(result[0])
    
    os.chdir(destination)
    try:
        shutil.rmtree(dest)
    except OSError as e:
        print("Error: %s : %s" % (dest, e.strerror))
        
    return local_name, local_df

# Example Usage
#check = extract_monash_df('tmp.zip')

def retrieve_monash_df(url, destination):
    
    # Create Soup
    page = requests.get(url)
    
    # Create Soup
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Find Download URL from Filename Class
    info = soup.select("[class~=filename]")
    
    # Parse Download Url
    download_url = info[0].get('href')
    
    # Download File
    urllib.request.urlretrieve("https://zenodo.org" + download_url, "tmp.zip")
    
# Example Usage
#retrieve_monash_df(df['URL'][1])


In [15]:
# Focus on just a few datasets
df = df[df['Frequency'].isin(['Daily', 'Monthly', 'Yearly'])]
df.head()

AttributeError: 'datatable.Frame' object has no attribute 'isin'

In [14]:
project_path = 'Users/jfarland/Documents/research/timeseries-benchmarks'
data_path = project_path + '/data'

os.getcwd()

'/Users/jfarland/Documents/research/timeseries-benchmarks'

In [90]:
import os
import shutil
import glob

from datetime import datetime
from distutils.util import strtobool

import pandas as pd


for index, row in tqdm(df.iterrows()):
    
    # Make sure current directory is data directory
    os.chdir(data_path)
    
    # Identify url from mapping file
    url = row.URL
    print(f'RETRIEVING URL: {url}')
    
    # Retrieve the file from the url
    retrieve_monash_df(url)
    
    print(f'PARSING AND EXTRACTING')
    local_name, local_df = extract_monash_df('tmp.zip')
    
    # create subdirectory
    os.makedirs(local_name)
    local_df.to_csv(local_name + '/' + local_name + '.csv', index=False)
    
    
 




0it [00:00, ?it/s]

RETRIEVING URL: https://zenodo.org/record/4656548
PARSING AND EXTRACTING
staging
/Users/jfarland/Documents/products/tidal-pulse/data
m4_daily_dataset.tsf


4227it [05:05, 13.85it/s]
1it [07:01, 421.91s/it]

RETRIEVING URL: https://zenodo.org/record/4654822


1it [08:48, 528.77s/it]


KeyboardInterrupt: 

In [24]:
from datetime import datetime
from distutils.util import strtobool

import pandas as pd


# Converts the contents in a .tsf file into a dataframe and returns it along with other meta-data of the dataset: frequency, horizon, whether the dataset contains missing values and whether the series have equal lengths
#
# Parameters
# full_file_path_and_name - complete .tsf file path
# replace_missing_vals_with - a term to indicate the missing values in series in the returning dataframe
# value_column_name - Any name that is preferred to have as the name of the column containing series values in the returning dataframe
def convert_tsf_to_dataframe(
    full_file_path_and_name,
    replace_missing_vals_with="NaN",
    value_column_name="series_value",
):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, "r", encoding="cp1252") as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"):  # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (
                                len(line_content) != 3
                            ):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if (
                                len(line_content) != 2
                            ):  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(
                                    strtobool(line_content[1])
                                )
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception(
                                "Missing attribute section. Attribute section must come before data."
                            )

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception(
                            "Missing attribute section. Attribute section must come before data."
                        )
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if len(series) == 0:
                            raise Exception(
                                "A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol"
                            )

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if numeric_series.count(replace_missing_vals_with) == len(
                            numeric_series
                        ):
                            raise Exception(
                                "All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series."
                            )

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(
                                    full_info[i], "%Y-%m-%d %H-%M-%S"
                                )
                            else:
                                raise Exception(
                                    "Invalid attribute type."
                                )  # Currently, the code supports only numeric, string and date types. Extend this as required.

                            if att_val is None:
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return (
            loaded_data,
            frequency,
            forecast_horizon,
            contain_missing_values,
            contain_equal_length,
        )


In [25]:
# Example of usage
loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("m1_yearly_dataset.tsf")

print(loaded_data)
print(frequency)
print(forecast_horizon)
print(contain_missing_values)
print(contain_equal_length)

    series_name start_timestamp  \
0            T1      1972-01-01   
1            T2      1974-01-01   
2            T3      1974-01-01   
3            T4      1974-01-01   
4            T5      1976-01-01   
..          ...             ...   
176        T177      1974-01-01   
177        T178      1973-01-01   
178        T179      1973-01-01   
179        T180      1975-01-01   
180        T181      1975-01-01   

                                          series_value  
0    [3600.0, 7700.0, 12300.0, 30500.0, 47390.0, 57...  
1    [12654.0, 22879.0, 34164.0, 49524.0, 64761.0, ...  
2    [2142.0, 12935.0, 19130.0, 30500.0, 48177.0, 5...  
3    [5774.0, 7650.0, 9271.0, 21447.0, 28998.0, 409...  
4    [432312.0, 569011.0, 862673.0, 1155640.0, 1439...  
..                                                 ...  
176  [290783.0, 285242.0, 293718.0, 295804.0, 29458...  
177  [11693.0, 11702.0, 11703.0, 11557.0, 11951.0, ...  
178  [8438.0, 8689.0, 8590.0, 8763.0, 8710.0, 8837....  
179  [55.

In [26]:
type(loaded_data)

pandas.core.frame.DataFrame

In [27]:
loaded_data.head()

,series_name,start_timestamp,series_value
0,T1,1972-01-01,"[3600.0, 7700.0, 12300.0, 30500.0, 47390.0, 57..."
1,T2,1974-01-01,"[12654.0, 22879.0, 34164.0, 49524.0, 64761.0, ..."
2,T3,1974-01-01,"[2142.0, 12935.0, 19130.0, 30500.0, 48177.0, 5..."
3,T4,1974-01-01,"[5774.0, 7650.0, 9271.0, 21447.0, 28998.0, 409..."
4,T5,1976-01-01,"[432312.0, 569011.0, 862673.0, 1155640.0, 1439..."


In [32]:
import os

#import sktime
#from sktime.utils.data_io import load_from_tsfile_to_dataframe
from sktime.datatypes import check_is_scitype

check_is_scitype(loaded_data)



#DATA_PATH = os.path.join(os.path.dirname(sktime.__file__), "datasets/data")

# train_x, train_y = load_from_tsfile_to_dataframe(
#     os.path.join(DATA_PATH, "ArrowHead/ArrowHead_TRAIN.ts")
# )
# test_x, test_y = load_from_tsfile_to_dataframe(
#     os.path.join(DATA_PATH, "ArrowHead/ArrowHead_TEST.ts")

TypeError: check_is_scitype() missing 1 required positional argument: 'scitype'

In [33]:
loaded_data


,series_name,start_timestamp,series_value
0,T1,1972-01-01,"[3600.0, 7700.0, 12300.0, 30500.0, 47390.0, 57..."
1,T2,1974-01-01,"[12654.0, 22879.0, 34164.0, 49524.0, 64761.0, ..."
2,T3,1974-01-01,"[2142.0, 12935.0, 19130.0, 30500.0, 48177.0, 5..."
3,T4,1974-01-01,"[5774.0, 7650.0, 9271.0, 21447.0, 28998.0, 409..."
4,T5,1976-01-01,"[432312.0, 569011.0, 862673.0, 1155640.0, 1439..."
...,...,...,...
176,T177,1974-01-01,"[290783.0, 285242.0, 293718.0, 295804.0, 29458..."
177,T178,1973-01-01,"[11693.0, 11702.0, 11703.0, 11557.0, 11951.0, ..."
178,T179,1973-01-01,"[8438.0, 8689.0, 8590.0, 8763.0, 8710.0, 8837...."
179,T180,1975-01-01,"[55.91, 54.7, 55.3, 55.75, 55.46, 55.37, 53.82..."


In [34]:
frequency

'yearly'

In [35]:
row = loaded_data.head(1)
row

,series_name,start_timestamp,series_value
0,T1,1972-01-01,"[3600.0, 7700.0, 12300.0, 30500.0, 47390.0, 57..."


In [51]:
from tqdm import tqdm

parsed_df = pd.DataFrame()

#freq = frequency
if frequency == 'yearly':
    freq = 'YS' #year start
elif frequency == 'quarterly':
    freq = 'QS' #quarter start
elif frequency == 'monthly':
    freq = 'MS'


for index,row in tqdm(loaded_data.iterrows()):
    
    name = row.series_name
    #print(name)
    values = row.series_value.tolist()
    length = len(values)
    start = row.start_timestamp
    #print(length)
    ds = pd.date_range(start, periods=length, freq=freq)
    
    series_df = pd.DataFrame({'unique_id':name,'ds':ds, 'values':values})
    parsed_df = pd.concat([parsed_df, series_df], axis=0)
    
parsed_df.head()
    
    
    
    
    




181it [00:00, 1246.58it/s]


,unique_id,ds,values
0,T1,1972-01-01,3600.0
1,T1,1973-01-01,7700.0
2,T1,1974-01-01,12300.0
3,T1,1975-01-01,30500.0
4,T1,1976-01-01,47390.0


In [84]:
def parse_monash_df(file):
    
    loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe(file)

    parsed_df = pd.DataFrame()

    #freq = frequency
    if frequency == 'yearly':
        freq = 'YS' #year start
    elif frequency == 'quarterly':
        freq = 'QS' #quarter start
    elif frequency == 'monthly':
        freq = 'MS'
    elif frequency == 'daily':
        freq = 'D'


    for index,row in tqdm(loaded_data.iterrows()):
        
        name = row.series_name
        #print(name)
        values = row.series_value.tolist()
        length = len(values)
        start = row.start_timestamp
        #print(length)
        ds = pd.date_range(start, periods=length, freq=freq)
        
        series_df = pd.DataFrame({'unique_id':name,'ds':ds, 'values':values})
        parsed_df = pd.concat([parsed_df, series_df], axis=0)
        
    return parsed_df

#parse_monash_df("m1_yearly_dataset.tsf").head()